In [1]:
import numpy as np
import matplotlib
matplotlib.use("pgf")
import matplotlib.pyplot as plt
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
BUF_LEN = 1024
FS = 48000

true_idxs = np.arange(0, BUF_LEN, 1)
demo_sine = np.sin((2 * np.pi / BUF_LEN) * 5 * true_idxs) * BUF_LEN/2

def tria(idxs):
    res = np.copy(idxs)
    l = len(res)
    for i in range(l):
        if i < l//2:
            res[i] *= 2
        else:
            res[i] = (res[i] * (-2)) + (l * 2)
    return res / max(res) * (l - 1)

def sine(idxs):
    l = len(idxs)
    w0 = 2 * np.pi / l
    return np.sin(w0 * idxs) * l/2 + (l/2 - 1)

In [3]:
tria_idxs = tria(true_idxs)
sine_idxs = sine(true_idxs)
demo_sine_tria = np.copy(demo_sine)
demo_sine_sine = np.copy(demo_sine)
for i in true_idxs:
    demo_sine_tria[i] = demo_sine[int(tria_idxs[i])] 
    demo_sine_sine[i] = demo_sine[int(sine_idxs[i])]

all_idxs = [true_idxs, tria_idxs, sine_idxs]
all_waves = [demo_sine, demo_sine_tria, demo_sine_sine]
labels = ["base function", "triangle function", "sine function"]

fig, axs = plt.subplots(3, 3, figsize=(6, 6))

for i, (idx, wave, label) in enumerate(zip(all_idxs, all_waves, labels)):
    axs[i, 0].plot(idx, label=label)
    axs[i, 0].set_xlabel("True index")
    axs[i, 0].set_ylabel("Computed index")
    axs[i, 0].grid()

    axs[i, 1].plot(wave, label=f"Sine indexed by {label}")
    axs[i, 1].set_xlabel("True index")
    axs[i, 1].set_ylabel("Computed value")
    axs[i, 1].grid()
    
    X = np.abs(np.fft.rfft(wave))
    axs[i, 2].plot(X / max(X), label=f"Sine indexed by {label}")
    axs[i, 2].set_xlabel("Frequency by buffer")
    axs[i, 2].set_ylabel("Normalized magnitude")
    axs[i, 2].grid()
    axs[i, 2].semilogx()

plt.tight_layout()

plt.savefig("indexer_types.pgf")
#!pip install tikzplotlib
#import tikzplotlib
#tikzplotlib.save("indexer_types.pgf")

In [4]:
plt.plot(true_idxs, sine(sine(sine(true_idxs))))
plt.xlabel("True index")
plt.ylabel("Computed index")
plt.grid()

In [5]:
plt.plot(tria(tria(tria(true_idxs))))
plt.xlabel("True index")
plt.ylabel("Computed index")
plt.grid()

In [6]:
plt.plot(sine(tria(sine(true_idxs))))
plt.xlabel("True index")
plt.ylabel("Computed index")
plt.grid()

In [8]:
import librosa as lr

true_idxs = np.arange(0, FS, 1)
tom_idxs = sine(tria(sine(true_idxs)))
demo_tom, _ = lr.load("./samples/tom16.wav", sr=FS, duration=1)
demo_tom_mod = np.copy(demo_tom)
for i in true_idxs:
    demo_tom_mod[i] = demo_tom[int(tom_idxs[i])]

all_idxs = [true_idxs, tom_idxs]
all_waves = [demo_tom, demo_tom_mod]
labels = ["original", "cascaded modulation"]

fig, axs = plt.subplots(2, 3, figsize=(6, 4))

for i, (idx, wave, label) in enumerate(zip(all_idxs, all_waves, labels)):
    
    # visually downsample data for smaller file size
    factor = 2
    vis_idx_y = [idx[n] for n in range(len(idx)) if n % factor == 0]
    vis_idx_x = np.arange(0, FS, factor)
    vis_wave_y = [wave[n] for n in range(len(wave)) if n % factor == 0]
    
    axs[i, 0].plot(vis_idx_x, vis_idx_y, label=label)
    axs[i, 0].set_xlabel("True index")
    axs[i, 0].set_ylabel("Computed index")
    axs[i, 0].grid()

    axs[i, 1].plot(vis_idx_x, vis_wave_y, label=f"Tom indexed by {label}")
    axs[i, 1].set_xlabel("True index")
    axs[i, 1].set_ylabel("Computed value")
    axs[i, 1].grid()
    
    X = np.abs(np.fft.rfft(wave))
    #X /= max(X)
    vis_X_x = np.arange(0, FS//2 + 1 + factor, factor)
    vis_X_y = np.append(X[0], [X[n] for n in range(len(X)) if n % factor == 0])
    axs[i, 2].plot(vis_X_x, vis_X_y, label=f"Tom indexed by {label}")
    #axs[i, 2].plot(vis_X_x, X, label=f"Tom indexed by {label}")
    axs[i, 2].set_xlabel("Frequency by buffer")
    axs[i, 2].set_ylabel("Normalized magnitude")
    axs[i, 2].grid()
    axs[i, 2].semilogx()

plt.tight_layout()

plt.savefig("tom_modulated.pgf")
#!pip install tikzplotlib
#import tikzplotlib
#tikzplotlib.save("tom_modulated.pgf")